<a href="https://colab.research.google.com/github/harryguiacorn/colab/blob/main/Seasonality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas matplotlib mplfinance plotly

# Monthly Seasonality

In [124]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta
import os
import mplfinance as mpf

class Model_MonthlyReturnsAnalyzer:
    def __init__(self, ticker_symbol, look_back_years):
        self.ticker_symbol = ticker_symbol
        self.look_back_years = look_back_years
        self.start_date = (
            datetime(datetime.now().year - look_back_years - 1, 12, 1)
        ).strftime("%Y-%m-%d")
        self.end_date = (
            datetime(datetime.now().year, 1, 1)
        ).strftime("%Y-%m-%d")
        self.csv_data = None
        self.grouped_data = None

    def download_data(self):
        self.csv_data = yf.download(
            self.ticker_symbol, start=self.start_date, end=self.end_date, interval="1mo"
        )
        self.csv_data["Open_Close_Return Pct"] = self.csv_data["Adj Close"].pct_change() * 100
        # print(csv_data)

    def calculate_average_monthly_returns(self):
        self.grouped_data = self.csv_data.groupby(self.csv_data.index.month)[
            "Open_Close_Return Pct"
        ].mean()
        # print(self.grouped_data)
        self.grouped_data = self.grouped_data.reset_index().rename(
            columns={"Date": "Month"}
        )
    def save_csv_to_folder(self, folder_path, filename, data, index=True):
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = os.path.join(folder_path, filename)
        data.to_csv(file_path, index=index)

class View_MonthlyReturnsAnalyzer:
    # def __init__(self):
    #   pass

    def create_candlestick_chart(self, csv_data, ticker_symbol):
      # Plot the candlestick chart
      mpf.plot(csv_data, type="candle", title=f"{ticker_symbol} Candlestick Chart", style="yahoo")

    def create_static_bar_plot(self, grouped_data, ticker_symbol, start_date, end_date):
        plt.figure(figsize=(10, 6))
        grouped_data.plot(
            x="Month", y="Open_Close_Return Pct", kind="bar", color="blue"
        )
        plt.title(
            f"Average Monthly Returns for {ticker_symbol} {start_date} - {end_date}"
        )
        plt.xlabel("Month")
        plt.ylabel("Average Monthly Return (%)")
        plt.show()
    def create_interactive_bar_plot(self, grouped_data, ticker_symbol, start_date, end_date):
        # Format the values to two decimal places
        grouped_data['Open_Close_Return Pct'] = grouped_data['Open_Close_Return Pct'].round(2)

        fig = px.bar(
            grouped_data,
            x="Month",
            y="Open_Close_Return Pct",
            title=f"Average Monthly Returns for {ticker_symbol} {start_date} - {end_date}",
        )
        fig = px.bar(
          grouped_data,
          x="Month",
          y="Open_Close_Return Pct",
          title=f"Average Monthly Returns for {ticker_symbol} {start_date} - {end_date}",
          labels={ # replaces default labels by column name
          "Open_Close_Return Pct": "Return Pct"
          },
        )

        # Add text annotations with two decimal places
        # fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
        fig.update_traces(texttemplate='%{y}', textposition='outside')

        # Update layout for better readability
        fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

        fig.update_layout(xaxis_title="Month", yaxis_title="Average Monthly Return (%)")
        fig.show()

class Controller_MonthlyReturnsAnalyzer:
    def __init__(self, ticker_symbol, look_back_years):
        self.ticker_symbol = ticker_symbol
        self.look_back_years = look_back_years
        self.model_MonthlyReturnsAnalyzer = Model_MonthlyReturnsAnalyzer(ticker_symbol, look_back_years)
        self.view_MonthlyReturnsAnalyzer = View_MonthlyReturnsAnalyzer()

    def analyze_and_save(self):
        # Define the folder path for saving CSV files
        folder_path = f"assets/{self.ticker_symbol.lower()}"
        data_filename = f"{self.ticker_symbol.lower()}_data_{self.look_back_years}yr.csv"
        grouped_filename = f"grouped_data_{self.look_back_years}yr.csv"
        self.model_MonthlyReturnsAnalyzer.download_data()
        self.model_MonthlyReturnsAnalyzer.calculate_average_monthly_returns()

        # Save CSV files to the specified folder path
        self.model_MonthlyReturnsAnalyzer.save_csv_to_folder(folder_path, data_filename, self.model_MonthlyReturnsAnalyzer.csv_data)
        self.model_MonthlyReturnsAnalyzer.save_csv_to_folder(folder_path, grouped_filename, self.model_MonthlyReturnsAnalyzer.grouped_data, False)

        # You can optionally call the visualization methods here as well
        # self.view_MonthlyReturnsAnalyzer.create_candlestick_chart(self.model_MonthlyReturnsAnalyzer.csv_data, self.ticker_symbol)
        # self.view_MonthlyReturnsAnalyzer.create_static_bar_plot(self.model_MonthlyReturnsAnalyzer.grouped_data, self.ticker_symbol, self.model_MonthlyReturnsAnalyzer.start_date, self.model_MonthlyReturnsAnalyzer.end_date)
        self.view_MonthlyReturnsAnalyzer.create_interactive_bar_plot(self.model_MonthlyReturnsAnalyzer.grouped_data, self.ticker_symbol, self.model_MonthlyReturnsAnalyzer.start_date, self.model_MonthlyReturnsAnalyzer.end_date)

if __name__ == "__main__":
    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "SPY", look_back_years = 5)
    analyzer.analyze_and_save()
    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "SPY", look_back_years = 10)
    analyzer.analyze_and_save()
    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "SPY", look_back_years = 15)
    analyzer.analyze_and_save()

    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "BTC-USD", look_back_years = 3)
    analyzer.analyze_and_save()
    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "BTC-USD", look_back_years = 5)
    analyzer.analyze_and_save()
    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "BTC-USD", look_back_years = 7)
    analyzer.analyze_and_save()
    analyzer = Controller_MonthlyReturnsAnalyzer(ticker_symbol = "BTC-USD", look_back_years = 9)
    analyzer.analyze_and_save()



[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


In [116]:
import plotly.express as px

# Sample data
data = pd.DataFrame({
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May'],
    'Value': [10, 15, 12, 18, 14]
})

# Create the bar chart
fig = px.bar(data, x='Month', y='Value')

# Add text annotations
fig.update_traces(texttemplate='%{y}', textposition='outside')

# Update layout for better readability
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Show the plot
fig.show()
